In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [3]:
df = yf.download('EURUSD=X', interval='15m', period='60d', multi_level_index=False)

C:\Users\Ido Sabach\AppData\Local\Temp\ipykernel_15568\3647498583.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('EURUSD=X', interval='15m', period='60d', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


In [4]:
df

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-08-25 23:00:00+00:00,1.161980,1.162385,1.161980,1.162385,0
2025-08-25 23:15:00+00:00,1.161845,1.161980,1.161845,1.161845,0
2025-08-25 23:30:00+00:00,1.161845,1.161980,1.161710,1.161710,0
2025-08-25 23:45:00+00:00,1.161710,1.161845,1.161710,1.161710,0
2025-08-26 00:00:00+00:00,1.165094,1.165094,1.161440,1.161575,0
...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0


In [5]:
df.to_csv('data/df60d.csv')

In [12]:
df = pd.read_csv('data/df60d.csv', index_col=0, parse_dates=True)

class DataLoad():
  def __init__(self, df):
    self.raw_data = df
    self.data = df

  def process_data(self):
    df = self.data.copy()

    df = df.rename(columns={'Close':'price'})
    df['returns'] = np.log(df['price'] / df['price'].shift(1))

    # convert column to date type
    df.index = pd.to_datetime(df.index)
    df['hour'] = df.index.hour
    df['date'] = df.index.date  

    conditions = [
      (df['hour'] >= 2) & (df['hour'] < 10),
      (df['hour'] >= 10) & (df['hour'] < 15),
      (df['hour'] >= 15) & (df['hour'] <= 23),
      (df['hour'] >= 0) & (df['hour'] < 2)
    ]

    choices = ['asia', 'london', 'ny', 'deadzone']
    df['sessions'] = np.select(conditions, choices, default='deadzone')

    df.dropna(inplace=True)
    self.data = df   
    return self.data

In [13]:
eur = DataLoad(df)

In [14]:
eur.data

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-08-25 23:00:00+00:00,1.161980,1.162385,1.161980,1.162385,0
2025-08-25 23:15:00+00:00,1.161845,1.161980,1.161845,1.161845,0
2025-08-25 23:30:00+00:00,1.161845,1.161980,1.161710,1.161710,0
2025-08-25 23:45:00+00:00,1.161710,1.161845,1.161710,1.161710,0
2025-08-26 00:00:00+00:00,1.165094,1.165094,1.161440,1.161575,0
...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0


In [15]:
eur.process_data()

,price,High,Low,Open,Volume,returns,hour,date,sessions
Datetime,,,,,,,,,
2025-08-25 23:15:00+00:00,1.161845,1.161980,1.161845,1.161845,0,-0.000116,23,2025-08-25,ny
2025-08-25 23:30:00+00:00,1.161845,1.161980,1.161710,1.161710,0,0.000000,23,2025-08-25,ny
2025-08-25 23:45:00+00:00,1.161710,1.161845,1.161710,1.161710,0,-0.000116,23,2025-08-25,ny
2025-08-26 00:00:00+00:00,1.165094,1.165094,1.161440,1.161575,0,0.002909,0,2025-08-26,deadzone
2025-08-26 00:15:00+00:00,1.165501,1.165501,1.164687,1.165094,0,0.000350,0,2025-08-26,deadzone
...,...,...,...,...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0,0.000116,21,2025-11-14,ny
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0,-0.000116,21,2025-11-14,ny
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0,0.000116,21,2025-11-14,ny


In [17]:
eur.data['sessions'].value_counts()

sessions
ny          2047
asia        1885
london      1180
deadzone     472
Name: count, dtype: int64